<a href="https://colab.research.google.com/github/CodeByAarish/Cognifyz-Technologies-Internship/blob/main/Task_1_Predict_Restaurant_Ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OBJECTIVE:**
# **Build a Machine Learning model to predict the aggregate rating of a restaurant based on other features.**

In [1]:
# Import important Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the Dataset
data = pd.read_csv('/content/Cognifyz Technologies.csv')
data.head(2)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591


In [3]:
df = data.copy()

# **Handling Missing Values and Duplicates | Feature Engineering**

In [4]:
df.isnull().sum()

,0
Restaurant ID,0
Restaurant Name,0
Country Code,0
City,0
Address,0
Locality,0
Locality Verbose,0
Longitude,0
Latitude,0
Cuisines,9


In [5]:
df.dropna(subset = ["Cuisines"], inplace = True)

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9542 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9542 non-null   int64  
 1   Restaurant Name       9542 non-null   object 
 2   Country Code          9542 non-null   int64  
 3   City                  9542 non-null   object 
 4   Address               9542 non-null   object 
 5   Locality              9542 non-null   object 
 6   Locality Verbose      9542 non-null   object 
 7   Longitude             9542 non-null   float64
 8   Latitude              9542 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9542 non-null   int64  
 11  Currency              9542 non-null   object 
 12  Has Table booking     9542 non-null   object 
 13  Has Online delivery   9542 non-null   object 
 14  Is delivering now     9542 non-null   object 
 15  Switch to order menu  9542

In [8]:
# Remove Unnacessary Columns
df = df.drop(columns = ["Restaurant ID", "Restaurant Name", "Address", "Locality", "Locality Verbose", "Longitude", "Latitude", "Average Cost for two", "Currency", "Is delivering now", "Switch to order menu", "Rating color", "Rating text"])
df.head(2)

,Country Code,City,Cuisines,Has Table booking,Has Online delivery,Price range,Aggregate rating,Votes
0,162,Makati City,"French, Japanese, Desserts",Yes,No,3,4.8,314
1,162,Makati City,Japanese,Yes,No,3,4.5,591


In [9]:
# Can be Encoded by Label Encoder
len(df["City"].unique())

140

In [10]:
# Can be Encoded by Label Encoding or Frequesncy Encoder
len(df["Cuisines"].unique())

1825

In [11]:
# Can be Encoded by One Hot Encoder
df["Has Table booking"].value_counts()

,count
Has Table booking,
No,8384
Yes,1158


In [12]:
# Can be Encoded by One Hot Encoder
df["Has Online delivery"].value_counts()

,count
Has Online delivery,
No,7091
Yes,2451


# **Building Machine Learning Model**

In [13]:
x = df.drop(columns = ["Aggregate rating"])
y = df["Aggregate rating"]

num_features = ["Country Code", "Price range", "Votes"]
cat_ohe_features = ["Has Table booking", "Has Online delivery"]
cat_le_features = ["City", "Cuisines"]

In [14]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for feature in cat_le_features:
    x[feature] = label_encoder.fit_transform(x[feature])

In [15]:
x.head()

,Country Code,City,Cuisines,Has Table booking,Has Online delivery,Price range,Votes
0,162,73,920,Yes,No,3,314
1,162,73,1111,Yes,No,3,591
2,162,75,1671,Yes,No,4,270
3,162,75,1126,No,No,4,365
4,162,75,1122,Yes,No,4,229


In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
cat_ohe_transformer = OneHotEncoder(drop = "first")

preprocessor = ColumnTransformer(
    [
        ("StandardScaler", num_transformer, num_features),
        ("OneHotEncoder", cat_ohe_transformer, cat_ohe_features)
    ], remainder = "passthrough"
)

In [17]:
# Splitting the Data into Training and Testing Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [18]:
pd.DataFrame(x_train).head()

,0,1,2,3,4,5,6
0,-0.301893,-0.882182,-0.346497,0.0,0.0,88.0,1699.0
1,-0.301893,0.218095,-0.105507,0.0,0.0,87.0,1626.0
2,0.215093,2.418648,-0.335130,0.0,0.0,120.0,1171.0
3,-0.301893,-0.882182,-0.360138,0.0,0.0,50.0,1275.0
4,-0.301893,0.218095,-0.148703,0.0,1.0,88.0,58.0


In [19]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(random_state=42),
    "Lasso": Lasso(random_state=42),
    "Elastic Net": ElasticNet(random_state=42),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=42),
    "SVR": SVR(),
    "XGBRegressor": XGBRegressor(random_state=42)
}

for name, model in models.items():
  model.fit(x_train, y_train) # Fitting the Training Data in Models

  y_train_pred = model.predict(x_train)  #Predicting from Training Data
  y_test_pred = model.predict(x_test)    #Predicting from Testing Data

  # Training and Testing Performance Matrix

  print("="*60)
  print(f"📘 Model: {name}")
  print("="*60)
  print("🔹 Training Metrics:")
  print(f"R2 Score: {r2_score(y_train, y_train_pred):.4f}")
  print(f"MSE: {mean_squared_error(y_train, y_train_pred):.4f}")
  print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.4f}")
  print(f"MAE: {mean_absolute_error(y_train, y_train_pred):.4f}")
  print(f"MAPE: {mean_absolute_percentage_error(y_train, y_train_pred):.4f}")
  print(f"Max Error: {max_error(y_train, y_train_pred):.4f}")
  print(f"Explained Variance: {explained_variance_score(y_train, y_train_pred):.4f}")

  print("\n🔹 Testing Metrics:")
  print(f"R2 Score: {r2_score(y_test, y_test_pred):.4f}")
  print(f"MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
  print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
  print(f"MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
  print(f"MAPE: {mean_absolute_percentage_error(y_test, y_test_pred):.4f}")
  print(f"Max Error: {max_error(y_test, y_test_pred):.4f}")
  print(f"Explained Variance: {explained_variance_score(y_test, y_test_pred):.4f}")
  print("="*60 + "\n")

📘 Model: Linear Regression
🔹 Training Metrics:
R2 Score: 0.2982
MSE: 1.6157
RMSE: 1.2711
MAE: 1.0570
MAPE: 2052966916437982.0000
Max Error: 5.0954
Explained Variance: 0.2982

🔹 Testing Metrics:
R2 Score: 0.3335
MSE: 1.5264
RMSE: 1.2355
MAE: 1.0196
MAPE: 2038186614153455.5000
Max Error: 3.9000
Explained Variance: 0.3340

📘 Model: Ridge
🔹 Training Metrics:
R2 Score: 0.2982
MSE: 1.6157
RMSE: 1.2711
MAE: 1.0570
MAPE: 2053117085069313.0000
Max Error: 5.0954
Explained Variance: 0.2982

🔹 Testing Metrics:
R2 Score: 0.3334
MSE: 1.5264
RMSE: 1.2355
MAE: 1.0196
MAPE: 2038341162299027.7500
Max Error: 3.8997
Explained Variance: 0.3340

📘 Model: Lasso
🔹 Training Metrics:
R2 Score: 0.0217
MSE: 2.2523
RMSE: 1.5008
MAE: 1.2092
MAPE: 2671281215863387.5000
Max Error: 3.1742
Explained Variance: 0.0217

🔹 Testing Metrics:
R2 Score: 0.0233
MSE: 2.2367
RMSE: 1.4956
MAE: 1.1999
MAPE: 2676749522869299.5000
Max Error: 3.2004
Explained Variance: 0.0234

📘 Model: Elastic Net
🔹 Training Metrics:
R2 Score: 0.0696


Model Performance Matrix

| Model                | R² Train | R² Test | RMSE Train | RMSE Test | MAE Train | MAE Test |     MAPE Train     |     MAPE Test      | MaxErr Train | MaxErr Test | ExpVar Train | ExpVar Test | Remark                          |
|----------------------|----------|---------|------------|-----------|-----------|----------|--------------------|--------------------|---------------|--------------|---------------|--------------|----------------------------------|
| Linear Regression     | 0.2982   | 0.3335  | 1.2711     | 1.2355    | 1.0570    | 1.0196   | 2.05e+15           | 2.04e+15           | 5.0954        | 3.9000       | 0.2982        | 0.3340       | ⚠️ Weak baseline, high MAPE     |
| Ridge                 | 0.2982   | 0.3334  | 1.2711     | 1.2355    | 1.0570    | 1.0196   | 2.05e+15           | 2.04e+15           | 5.0954        | 3.8997       | 0.2982        | 0.3340       | ⚠️ Same as Linear Regression     |
| Lasso                 | 0.0217   | 0.0233  | 1.5008     | 1.4956    | 1.2092    | 1.1999   | 2.67e+15           | 2.68e+15           | 3.1742        | 3.2004       | 0.0217        | 0.0234       | ❌ Underfit model                |
| Elastic Net           | 0.0696   | 0.0746  | 1.4636     | 1.4557    | 1.1832    | 1.1715   | 2.61e+15           | 2.61e+15           | 3.2185        | 3.1501       | 0.0696        | 0.0747       | ❌ Still underfitting            |
| KNN Regressor         | 0.7597   | 0.6533  | 0.7438     | 0.8910    | 0.4648    | 0.5664   | 6.07e+14           | 7.42e+14           | 3.4000        | 4.3000       | 0.7612        | 0.6551       | ✅ Good generalization           |
| Decision Tree         | 0.9986   | 0.9163  | 0.0572     | 0.4379    | 0.0128    | 0.2815   | 0.0043             | 0.0848             | 0.8500        | 2.3000       | 0.9986        | 0.9163       | ⚠️ Overfitting risk              |
| Random Forest         | 0.9928   | 0.9536  | 0.1291     | 0.3261    | 0.0824    | 0.2139   | 0.0254             | 0.0658             | 0.8652        | 1.8020       | 0.9928        | 0.9536       | ✅ Excellent performer           |
| Gradient Boosting     | 0.9605   | 0.9576  | 0.3015     | 0.3115    | 0.1987    | 0.2066   | 1.75e+13           | 1.77e+13           | 2.0520        | 1.6761       | 0.9605        | 0.9577       | ✅ Top accuracy (slightly over)  |
| AdaBoost Regressor    | 0.9431   | 0.9439  | 0.3620     | 0.3584    | 0.2457    | 0.2447   | 0.0740             | 0.0743             | 1.7273        | 1.5995       | 0.9435        | 0.9440       | ✅ Balanced, solid performer     |
| SVR                   | -0.0934  | -0.0999 | 1.5866     | 1.5870    | 1.0622    | 1.0645   | 3.21e+15           | 3.22e+15           | 3.7735        | 3.6943       | 0.0339        | 0.0329       | ❌ Worst overall performance     |
| XGBRegressor          | 0.9821   | 0.9552  | 0.2030     | 0.3201    | 0.1312    | 0.2133   | 9.75e+12           | 1.13e+13           | 1.2077        | 1.7416       | 0.9821        | 0.9553       | 🔥 Best overall performance      |


# **Hyperperimeter Tuning**
- Decision Tree Regressor
- Random Forest Regressor
- Gradient Boosting Regressor
- AdaBoost Regressor
- XGBoost Regressor

In [20]:
# Decision Tree Parameters
decisiontree_params = {
    "criterion":["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "splitter":["best", "random"],
    "max_depth" : [1,2,3,5,7,8,10,20,30,40,50,100,200,500,1000, None],
    "max_features": ["sqrt", "log2"],
    "random_state":[0, 42, None]
}
# Random Forest Parameters
randomforest_params = {
    "n_estimators": [100, 200, 300, 500, 1000],
    "max_depth": [5, 10, 15, 20, None],
    "criterion": ["squared_error", "absolute_error", "poisson"],
    "max_features": ["sqrt", "log2", None],
    "min_samples_split": [2, 5, 10, 15, 20]
}
# Gradient Boosting Parameters
gradientboost_params = {
    "loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "learning_rate": [0.001,0.0001, 0.01, 0.1, 1, 10],
    "n_estimators": [100,200,300,400,500,600,700,800,900,1000,2000],
    "subsample": [0,0.01,0.001,0.0001,0.5,1,0.8,0.4],
    "criterion":["friedman_mse", "squared_error"],
    "random_state": [0, 42, None],
    "max_features": ["sqrt", "log2"]
}
#Adaboost Parameters
adaboost_params = {
    "n_estimators": [50, 100, 200, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.3, 0.5, 1.0],
    "loss": ["linear", "square", "exponential"],
    "estimator": [DecisionTreeRegressor(max_depth=d) for d in [2, 3, 4, 5, 6]]
}
#XGBoost Parameters
xgboost_params = {
    "n_estimators": [100, 200, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.3],
    "max_depth": [3, 4, 5, 6, 7, 8],
    "min_child_weight": [1, 3, 5, 7],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "gamma": [0, 0.1, 0.3, 0.5, 1],
    "reg_alpha": [0, 0.01, 0.1, 1],  # L1 regularization
    "reg_lambda": [0.1, 1, 10]       # L2 regularization
}

In [21]:
# Models list for Hyperperimeter Tuning using RandomizedSearch CV
randomcv_models = [("Decision Tree Regressor", DecisionTreeRegressor(random_state = 42), decisiontree_params),
                   ("Random Forest Regressor", RandomForestRegressor(random_state = 42), randomforest_params),
                   ("Gradient Boosting Regressor", GradientBoostingRegressor(random_state = 42), gradientboost_params),
                   ("AdaBoost Regressor", AdaBoostRegressor(random_state = 42), adaboost_params),
                   ("XG Boost Regressor", XGBRegressor(random_state = 42), xgboost_params)
                   ]

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from time import time

# Dictionary to store best parameters
best_params = {}

# Loop over each model and apply RandomizedSearchCV
for name, model, params in randomcv_models:
    print("="*80)
    print(f"🔍 Running RandomizedSearchCV for: {name}")
    print("="*80)

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        scoring='r2',
        cv=5,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    print("🔍 Starting Hyperparameter Tuning...")
    start_time = time()

    random_search.fit(x_train, y_train)
    end_time = time()
    print("✅ Tuning completed.")
    print(f"🕒 Time taken: {round(end_time - start_time, 2)} seconds")

    # Save best params
    best_params[name] = random_search.best_params_

    print(f"\n✅ Best Parameters for {name}:")
    print(random_search.best_params_)
    print("-"*80)

# Show all best parameters
print("\n🎯 All Best Hyperparameters Summary:")
for name, params in best_params.items():
    print(f"{name} ➤ {params}")


🔍 Running RandomizedSearchCV for: Decision Tree Regressor
🔍 Starting Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
✅ Tuning completed.
🕒 Time taken: 37.64 seconds

✅ Best Parameters for Decision Tree Regressor:
{'splitter': 'best', 'random_state': None, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'squared_error'}
--------------------------------------------------------------------------------
🔍 Running RandomizedSearchCV for: Random Forest Regressor
🔍 Starting Hyperparameter Tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
✅ Tuning completed.
🕒 Time taken: 11533.11 seconds

✅ Best Parameters for Random Forest Regressor:
{'n_estimators': 1000, 'min_samples_split': 20, 'max_features': None, 'max_depth': 20, 'criterion': 'squared_error'}
--------------------------------------------------------------------------------
🔍 Running RandomizedSearchCV for: Gradient Boosting Regressor
🔍 Starting Hyperparameter Tuning...


In [22]:
models = {
    "Decision Tree Regressor": DecisionTreeRegressor(splitter = 'best',
                                                     random_state =  None,
                                                     max_features =  'sqrt',
                                                     max_depth =  10,
                                                     criterion =  'squared_error'
),
    "Random Forest Regressor": RandomForestRegressor(n_estimators = 1000,
                                                     min_samples_split =  20,
                                                     max_features =  None,
                                                     max_depth =  20,
                                                     criterion =  'squared_error'
                                                     ),
    "Gradient Boosting Regressor": GradientBoostingRegressor(subsample =  1,
                                                             random_state =  None,
                                                             n_estimators =  1000,
                                                             max_features =  'sqrt',
                                                             loss =  'huber',
                                                             learning_rate =  0.1,
                                                             criterion =  'friedman_mse'
),
    "AdaBoost Regressor": AdaBoostRegressor(n_estimators =  100,
                                            loss =  'exponential',
                                            learning_rate =  0.05,
                                            estimator =  DecisionTreeRegressor(max_depth=6)
                                            ),
    "XGBRegressor": XGBRegressor(subsample =  0.7,
                                 reg_lambda =  10,
                                 reg_alpha =  0.01,
                                 n_estimators = 100,
                                 min_child_weight = 1,
                                 max_depth =  8,
                                 learning_rate =  0.1,
                                 gamma =  0.1,
                                 colsample_bytree =  1.0
                                 )}

for name, model in models.items():
  model.fit(x_train, y_train)
  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)

  # Training and Testing Performance Matrix

  print("="*60)
  print(f"📘 Model: {name}")
  print("="*60)
  print("🔹 Training Metrics:")
  print(f"R2 Score: {r2_score(y_train, y_train_pred):.4f}")
  print(f"MSE: {mean_squared_error(y_train, y_train_pred):.4f}")
  print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.4f}")
  print(f"MAE: {mean_absolute_error(y_train, y_train_pred):.4f}")
  print(f"MAPE: {mean_absolute_percentage_error(y_train, y_train_pred):.4f}")
  print(f"Max Error: {max_error(y_train, y_train_pred):.4f}")
  print(f"Explained Variance: {explained_variance_score(y_train, y_train_pred):.4f}")

  print("\n🔹 Testing Metrics:")
  print(f"R2 Score: {r2_score(y_test, y_test_pred):.4f}")
  print(f"MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
  print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
  print(f"MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
  print(f"MAPE: {mean_absolute_percentage_error(y_test, y_test_pred):.4f}")
  print(f"Max Error: {max_error(y_test, y_test_pred):.4f}")
  print(f"Explained Variance: {explained_variance_score(y_test, y_test_pred):.4f}")
  print("="*60 + "\n")

📘 Model: Decision Tree Regressor
🔹 Training Metrics:
R2 Score: 0.9493
MSE: 0.1167
RMSE: 0.3416
MAE: 0.2084
MAPE: 28447756313427.6602
Max Error: 3.1322
Explained Variance: 0.9493

🔹 Testing Metrics:
R2 Score: 0.9331
MSE: 0.1532
RMSE: 0.3914
MAE: 0.2454
MAPE: 22091991765201.9805
Max Error: 3.4000
Explained Variance: 0.9331

📘 Model: Random Forest Regressor
🔹 Training Metrics:
R2 Score: 0.9746
MSE: 0.0584
RMSE: 0.2416
MAE: 0.1563
MAPE: 0.0482
Max Error: 1.4563
Explained Variance: 0.9746

🔹 Testing Metrics:
R2 Score: 0.9577
MSE: 0.0968
RMSE: 0.3112
MAE: 0.2037
MAPE: 0.0630
Max Error: 1.7873
Explained Variance: 0.9577

📘 Model: Gradient Boosting Regressor
🔹 Training Metrics:
R2 Score: 0.9680
MSE: 0.0737
RMSE: 0.2714
MAE: 0.1747
MAPE: 27874948237352.2891
Max Error: 1.9548
Explained Variance: 0.9681

🔹 Testing Metrics:
R2 Score: 0.9599
MSE: 0.0919
RMSE: 0.3032
MAE: 0.2035
MAPE: 27086352475651.2930
Max Error: 1.7094
Explained Variance: 0.9600

📘 Model: AdaBoost Regressor
🔹 Training Metrics:
R2

Performance Matrix after Hyperperimeter Tuning

| Model                   | R² Train | R² Test | RMSE Train | RMSE Test | MAE Train | MAE Test |     MAPE Train     |     MAPE Test      | MaxErr Train | MaxErr Test | ExpVar Train | ExpVar Test | Remark                           |
|-------------------------|----------|---------|------------|-----------|-----------|----------|--------------------|--------------------|---------------|--------------|---------------|--------------|----------------------------------|
| Decision Tree Regressor | 0.9472   | 0.9252  | 0.3486     | 0.4137    | 0.2184    | 0.2557   | 2.31e+13           | 3.82e+13           | 3.4470        | 4.2000       | 0.9472        | 0.9253       | ⚠️ Good accuracy, mild overfit    |
| Random Forest Regressor | 0.9747   | 0.9577  | 0.2415     | 0.3111    | 0.1563    | 0.2036   | 0.0482             | 0.0629             | 1.4400        | 1.7887       | 0.9747        | 0.9578       | ✅ Excellent generalization       |
| Gradient Boosting       | 0.9682   | 0.9596  | 0.2704     | 0.3041    | 0.1745    | 0.2041   | 2.93e+13           | 2.98e+13           | 1.9122        | 1.7360       | 0.9683        | 0.9597       | ✅ Very high performance          |
| AdaBoost Regressor      | 0.9557   | 0.9544  | 0.3195     | 0.3233    | 0.2107    | 0.2141   | 0.0641             | 0.0656             | 1.8306        | 1.7122       | 0.9560        | 0.9545       | ✅ Balanced, good performer       |
| XGBRegressor            | 0.9719   | 0.9594  | 0.2544     | 0.3049    | 0.1657    | 0.2008   | 5.14e+12           | 5.24e+12           | 1.5525        | 1.6393       | 0.9719        | 0.9594       | 🔥 Best overall performance       |


🥇 XGBoost Regressor wins the race!

In [59]:
# Save model and preprocessor
import joblib
joblib.dump(preprocessor, 'preprocessor.pkl')
joblib.dump(model, 'XGBRegressor.pkl')

['XGBRegressor.pkl']

In [36]:
joblib.dump(label_encoder, 'le_city.pkl')   # LabelEncoder for City
joblib.dump(label_encoder, 'le_cuisine.pkl')  # LabelEncoder for Cuisines

['le_cuisine.pkl']

In [33]:
files.download('preprocessor.pkl')
files.download('XGBRegressor.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
files.download('le_city.pkl')
files.download('le_cuisine.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
from google.colab import files
uploaded = files.upload()

Saving XGBRegressor.pkl to XGBRegressor (4).pkl
Saving le_city.pkl to le_city (3).pkl
Saving le_cuisine.pkl to le_cuisine (3).pkl
Saving preprocessor.pkl to preprocessor (4).pkl
